In [1]:
from uniswap_liquidity import (get_factory, get_token_decimals, abi, 
                               UniV2Router, UniV3Router, UniV3Fee, SushiRouter,
                               token_base, token_dict) #see uniswap_liquidity.py
import os
import logging
import pandas as pd
from datetime import datetime
from web3 import Web3
from multicall import Call, Multicall

# The dafault RPC from ethersjs, change it if it doesn't work: https://infura.io/docs
RPC_Endpoint = 'https://mainnet.infura.io/v3/84842078b09946638c03157f83405213'



In [2]:
def main(w3):
    pools = {}

    UniFactory2 = get_factory(w3, abi, UniV2Router)
    logging.info(f'UniFactory2: {UniFactory2}')
    UniFactory3 = get_factory(w3,abi, UniV3Router)
    logging.info(f'UniFactory3: {UniFactory3}')
    SushiFactory = get_factory(w3,abi, SushiRouter)
    logging.info(f'SushiRouter: {SushiRouter}')

    for symbol0 in token_base:
        token0 = Web3.toChecksumAddress(token_base[symbol0])
        base_decimals = get_token_decimals(w3, abi, token0, symbol0)

        logging.info(f'pools count= {len(pools)}..')

        for symbol1 in token_dict:
            token1 = Web3.toChecksumAddress(token_dict[symbol1])
            token_decimals = get_token_decimals(w3, abi, token1, symbol1)

            pool_uni2 = w3.eth.contract(address=UniFactory2, abi=abi).functions.getPair(token0,token1).call()
            pools[pool_uni2] = {'token'        :symbol1,
                                'token_decimals':token_decimals,
                                'base_token'    :symbol0,
                                'base_decimals' :base_decimals,
                                'version'       :'Uniswap V2',
                                'fee' : None,}
            pool_sushi = w3.eth.contract(address=SushiFactory, abi=abi).functions.getPair(token0,token1).call()
            pools[pool_sushi] = {'token'       :symbol1,
                                 'token_decimals':token_decimals,
                                 'base_token'    :symbol0,
                                 'base_decimals' :base_decimals,
                                 'version'       :'Sushiswap',
                                 'fee' : None,}            

            multi_getPool = Multicall([
                                Call(UniFactory3, ['getPool(address,address,uint24)(address)', token0,token1,x], [[x, Web3.toChecksumAddress]]) for x in UniV3Fee
                                ]
                                ,_w3 = w3)

            multi_getPool = multi_getPool()

            pools.update({ multi_getPool[fee]:{'token':symbol1,
                                               'token_decimals':token_decimals,
                                               'base_token':symbol0,
                                               'base_decimals':base_decimals,
                                               'version':'Uniswap V3',
                                               'fee' : fee/10**6}
                          for fee in multi_getPool})

    logging.info(f'pools count= {len(pools)}')

    for pool in list(pools.keys()):
        if pool=='0x0000000000000000000000000000000000000000':
            pools.pop(pool)

    multi_balanceOwn = Multicall([Call(token_dict[pools[x]['token']],
                                      ['balanceOf(address)(uint256)', x],
                                      [[x, None]]
                                     ) for x in pools
                                ]
                                ,_w3 = w3)

    multi_balanceOwn = multi_balanceOwn()


    multi_balanceBase = Multicall([Call(token_base[pools[x]['base_token']],
                                      ['balanceOf(address)(uint256)', x],
                                      [[x, None]]
                                     ) for x in pools
                                ]
                                ,_w3 = w3)

    multi_balanceBase = multi_balanceBase()

    {pools[x].update({  'OwnBalance'           : multi_balanceOwn[x]*10**(-pools[x]['token_decimals']),
                        pools[x]['base_token'] : multi_balanceBase[x]*10**(-pools[x]['base_decimals']),
                     })
         for x in multi_balanceBase}

    logging.info(f'pools final count= {len(pools)}')

    return pools

In [3]:
if __name__ == '__main__':
    logging.basicConfig(
        format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
        level=os.environ.get('LOGLEVEL', 'INFO').upper(),
        datefmt='%Y-%m-%d %H:%M:a%S',
        )
    w3_eth = Web3(Web3.HTTPProvider(RPC_Endpoint, request_kwargs={'timeout': 20}))
    logging.info (f'Ethereum connected: {w3_eth.isConnected()}')

    ret  = main(w3_eth)


2022-05-28 19:25:a42 <module> INFO Ethereum connected: True
2022-05-28 19:25:a42 main INFO UniFactory2: 0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f
2022-05-28 19:25:a42 main INFO UniFactory3: 0x1F98431c8aD98523631AE4a59f267346ea31F984
2022-05-28 19:25:a43 main INFO SushiRouter: 0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F
2022-05-28 19:25:a43 main INFO pools count= 0..
2022-05-28 19:26:a21 main INFO pools count= 118..
2022-05-28 19:26:a53 main INFO pools count= 152..
2022-05-28 19:27:a27 main INFO pools count= 233..
2022-05-28 19:28:a01 main INFO pools count= 286
2022-05-28 19:28:a03 main INFO pools final count= 285


In [4]:
df = pd.DataFrame.from_dict(ret, orient = 'index').reset_index().rename(columns = {'index':'pool'})
df['batchtime'] = datetime.utcnow()
display(df)

,pool,token,token_decimals,base_token,base_decimals,version,fee,OwnBalance,WBTC,USDC,DAI,WETH,batchtime
0,0xBb2b8038a1640196FbE3e38816F3e67Cba72D940,WETH,18,WBTC,8,Uniswap V2,NaN,5.391142e+03,330.804781,NaN,NaN,NaN,2022-05-28 07:28:03.914232
1,0xCEfF51756c56CeFFCA006cD410B03FFC46dd3a58,WETH,18,WBTC,8,Sushiswap,NaN,1.627775e+04,999.210980,NaN,NaN,NaN,2022-05-28 07:28:03.914232
2,0xe6ff8b9A37B0fab776134636D9981Aa778c4e718,WETH,18,WBTC,8,Uniswap V3,0.0001,6.649689e-01,0.011510,NaN,NaN,NaN,2022-05-28 07:28:03.914232
3,0x4585FE77225b41b697C938B018E2Ac67Ac5a20c0,WETH,18,WBTC,8,Uniswap V3,0.0005,4.112066e+04,800.196383,NaN,NaN,NaN,2022-05-28 07:28:03.914232
4,0xCBCdF9626bC03E24f779434178A73a0B4bad62eD,WETH,18,WBTC,8,Uniswap V3,0.0030,7.923558e+04,2028.720613,NaN,NaN,NaN,2022-05-28 07:28:03.914232
...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,0x055CEDfe14BCE33F985C41d9A1934B7654611AAC,USDT,6,DAI,18,Sushiswap,NaN,1.659718e+02,NaN,NaN,1.695764e+02,NaN,2022-05-28 07:28:03.914232
281,0x6f48ECa74B38d2936B02ab603FF4e36A6C0E3A77,USDT,6,DAI,18,Uniswap V3,0.0005,1.297108e+06,NaN,NaN,2.649405e+05,NaN,2022-05-28 07:28:03.914232
282,0x4773E2C1c0B400a16DfeC4ca6E305141859a5542,USDT,6,DAI,18,Uniswap V3,0.0030,2.000000e-06,NaN,NaN,9.000000e+00,NaN,2022-05-28 07:28:03.914232
283,0x67A6C35f48aCEd972c0BC9168ec3Eb027B52BF7E,FXS,18,DAI,18,Uniswap V3,0.0001,3.000000e-18,NaN,NaN,2.000000e-18,NaN,2022-05-28 07:28:03.914232
